In [ ]:
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
import scipy.fftpack
%matplotlib inline

In [ ]:
from analysis.lib.nv import detection
folder= tb.latest_data()
# folder = "D:/measuring/data/20170308/105316_PQ_single_click_ent_111no2_SIL2_phase_stab"
a = m2.M2Analysis(folder)


In [ ]:
# stitch the different part and files together, make and save a plot of the data.
s = []
u_1 = []
v_1 = []

pid_counts_1 = a.g['adwindata']['pid_counts'].value
sample_counts_1 = a.g['adwindata']['sampling_counts'].value
max_repetitions = a.g['adwindata']['completed_reps'].value

delay = (a.g.attrs['count_int_cycles']/(a.g.attrs['cycle_duration']*100))
pid_cycles = a.g.attrs['pid_points']
sample_cycles = a.g.attrs['sample_points']
print max_repetitions

# stitch
for i in xrange(max_repetitions):
    for j in xrange(pid_cycles):
#             s.append(setpoint)
         u_1.append(pid_counts_1[j+i*pid_cycles])
         v_1.append(None)
    for k in xrange(sample_cycles):
#            s.append(setpoint)
        u_1.append(None)
        v_1.append(sample_counts_1[k+i*sample_cycles])


t = np.arange(0, (len(v_1)*delay/10), (0.1*delay))


fig = plt.figure(figsize=(17,6))
ax = plt.subplot(211)
# plt.plot(t, s, 'y')
plt.plot(t, u_1, 'r')
plt.plot(t, v_1, 'b')
plt.title('Counts ZPL detector 1 {0}'.format(folder))
ax.set_xlabel('elapsed time (milliseconds)')
ax.set_ylabel('counts')


fig.savefig(os.path.join(folder, 'trace_counts.png'))

In [ ]:
zoom_1 = 80*(pid_cycles+sample_cycles)
zoom_2 = 90*(pid_cycles+sample_cycles)

fig = plt.figure(figsize=(17,6))
ax = plt.subplot(211)
# plt.plot(t[0:zoom], s[0:zoom], 'y')
plt.plot(t[zoom_1:zoom_2], u_1[zoom_1:zoom_2], 'r')
plt.plot(t[zoom_1:zoom_2], v_1[zoom_1:zoom_2], 'b')

plt.title('Counts ZPL detector 1'.format(folder))
ax.set_xlabel('elapsed time (milliseconds)')
ax.set_ylabel('counts')

fig.savefig(os.path.join(folder, 'trace_counts_zoomed.png'))

In [ ]:
# fourier transform of the sample part


pid_counts_1 = a.g['adwindata']['pid_counts'].value
sample_counts_1 = a.g['adwindata']['sampling_counts'].value
max_repetitions = a.g['adwindata']['completed_reps'].value

delay = (a.g.attrs['count_int_cycles']/(a.g.attrs['cycle_duration']*100))
pid_cycles = a.g.attrs['pid_points']
sample_cycles = a.g.attrs['sample_points']

# stitch
v_1 = []
for i in xrange(max_repetitions):
    for j in xrange(pid_cycles):
         v_1.append(pid_counts_1[j+i*pid_cycles])
    for k in xrange(sample_cycles):
        v_1.append(sample_counts_1[k+i*sample_cycles])
yf = scipy.fftpack.fft(v_1)

# sample spacing
xf = np.linspace(0, 1.0/(2.0*delay*10**-4), (sample_cycles+pid_cycles)*max_repetitions/2)

index_max = [i for i,v in enumerate(yf) if v > 40000]
frq_max = [x * (1.0/(delay*10**-4)) / ((sample_cycles+pid_cycles)*max_repetitions) for x in index_max]
print 'peak frequencies are:'
print frq_max

norm1 = yf / np.linalg.norm(yf)

fig, ax = plt.subplots()
plt.ylim([0,100000])
ax.plot(xf, yf[:len(yf)/2])
plt.title('FFT {0}'.format(folder))
ax.set_xlabel('frequency (Hz)')
ax.set_ylabel('Amplitude (a.u.)')
# ax.set_yticklabels([])
fig.savefig(os.path.join(folder, 'fft.png'))